UCF Crime Detection Deep Learning Analysis

Gather data, determine the method of data collection and provenance of the data:

I grabbed the UCF Crime Dataset from Kaggle.

Identify a Deep Learning Problem:

I wanted to train a deep learning model to help solve a problem in the world. The problem I chose to focus on was crime. I imported the UCF Crime Dataset, a dataset that is split into a Test and Train folder each with label folders of Abuse, Arrest, Arson, Assault, Burglary, Explosion, Fighting, Normal Videos, Road Accidents, Robbery, Shooting, Shoplifting, Stealing, and Vandalism. Each of these label folders contains a series of png images that are captured from video surveillance of the denoted crime or action occuring. 

In [ ]:
# import necessary libraries
import os
import gc
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras import backend as K
from keras.models import Sequential
from keras.layers import (BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D, ReLU)
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
warnings.filterwarnings("ignore")

# set paths for train and test data
path_train = '../input/ucf-crime-dataset/Train'
path_test = '../input/ucf-crime-dataset/Test'

In [ ]:
# function to load and visualize images
def load_and_visualize_images(image_paths, target_size=(96, 96)):
    fig, axes = plt.subplots(1, len(image_paths), figsize=(20, 20))
    for img_path, ax in zip(image_paths, axes):
        img = load_img(img_path, target_size=target_size)
        ax.imshow(img)
        ax.axis('off')
    plt.show()
    
# example images for visualization
example_image_paths = [os.path.join(path_test, 'Vandalism', 'Vandalism007_x264_0.png'), os.path.join(path_test, 'Arrest', 'Arrest001_x264_0.png')]
load_and_visualize_images(example_image_paths)

In [ ]:
# image data generator parameters
img_gen_params = {
    "rescale": 1.0 / 255,
    "samplewise_center": True,
    "samplewise_std_normalization": True,
    "horizontal_flip": True,
    "vertical_flip": True
}

# create image data generator
img_gen = ImageDataGenerator(**img_gen_params)
IMAGE_SHAPE = (96, 96, 3)
batch_size = 32

# prepare train data generator
img_flow_train = img_gen.flow_from_directory(
    path_train,
    target_size=IMAGE_SHAPE[:2],
    batch_size=batch_size,
    class_mode='categorical'
)

# prepare test data generator
img_flow_test = img_gen.flow_from_directory(
    path_test,
    target_size=IMAGE_SHAPE[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# visualize pixel intensity distribution
sample_img = load_img(example_image_paths[0], target_size=IMAGE_SHAPE[:2])
sample_img_array = img_to_array(sample_img)
plt.figure(figsize=(10, 6))
plt.hist(sample_img_array.flatten(), bins=50)
plt.title('Pixel Intensity Distribution')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# define cnn model
kernel_size = (5, 5)
filters = (32, 64, 128)
drop_prob_conv = 0.3
drop_prob_dense = 0.3
model = Sequential([
    Conv2D(filters[0], kernel_size, padding="same", kernel_initializer='he_normal', input_shape=IMAGE_SHAPE),
    BatchNormalization(),
    ReLU(),
    Conv2D(filters[0], kernel_size, padding="same", kernel_initializer='he_normal'),
    BatchNormalization(),
    ReLU(),
    MaxPooling2D(),
    Dropout(drop_prob_conv),
    Conv2D(filters[1], kernel_size, padding="same", kernel_initializer='he_normal'),
    BatchNormalization(),
    ReLU(),
    Conv2D(filters[1], kernel_size, padding="same", kernel_initializer='he_normal'),
    BatchNormalization(),
    ReLU(),
    MaxPooling2D(),
    Dropout(drop_prob_conv),
    Conv2D(filters[2], kernel_size, padding="same", kernel_initializer='he_normal'),
    BatchNormalization(),
    ReLU(),
    Conv2D(filters[2], kernel_size, padding="same", kernel_initializer='he_normal'),
    BatchNormalization(),
    ReLU(),
    MaxPooling2D(),
    Dropout(drop_prob_conv),
    Flatten(),
    BatchNormalization(),
    ReLU(),
    Dropout(drop_prob_dense),
    Dense(256),
    BatchNormalization(),
    ReLU(),
    Dropout(drop_prob_dense),
    Dense(len(img_flow_train.class_indices), activation="softmax")
])

# compile model
model.compile(Adam(0.01), loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# define callbacks
early_stopping = EarlyStopping(monitor="val_accuracy", patience=3, verbose=1, mode='max')
lr_decay = ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=1, min_lr=1e-5)
# train model
print("Training the model...")
history = model.fit(
    img_flow_train,
    steps_per_epoch=img_flow_train.samples // batch_size,
    epochs=1,
    verbose=1,
    validation_data=img_flow_test,
    validation_steps=img_flow_test.samples // batch_size,
    callbacks=[lr_decay, early_stopping]
)
print("Done!")

In [ ]:
# predict on test data
val_steps = img_flow_test.samples // batch_size
y_test_pred = model.predict(img_flow_test, steps=val_steps)
y_test_true = img_flow_test.classes[:len(y_test_pred)]

# calculate accuracy
accuracy = np.equal(np.argmax(y_test_pred, axis=1), y_test_true).sum() / y_test_pred.shape[0]
print(f"Test accuracy: {accuracy:.3f}.")

In [ ]:
# plot roc curve
fpr, tpr, _ = roc_curve(y_test_true, y_test_pred[:, 1])
auc_val = auc(fpr, tpr)
print(f"Test AUC: {auc_val:.3f}.")
plt.figure()
plt.plot([0, 1], [0, 1], "k--")
plt.plot(fpr, tpr, label=f"ACC={accuracy:.4f}, AUC={auc_val:.4f}")
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.title("ROC curve")
plt.legend(loc="best")
plt.show()

In [ ]:
# plot precision-recall curve
precision, recall, _ = precision_recall_curve(y_test_true, y_test_pred[:, 1])
average_precision = average_precision_score(y_test_true, y_test_pred[:, 1])
print(f"Average precision score: {average_precision:.3f}.")
plt.figure()
plt.step(recall, precision, where='post', label=f'AP={average_precision:.4f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve')
plt.legend(loc="best")
plt.show()

In [ ]:
# collect garbage
gc.collect()

# save predictions to csv
file_names = img_flow_test.filenames
y_pred_df = pd.DataFrame({"file_name": file_names, "label": np.argmax(y_test_pred, axis=1)})
y_pred_df.to_csv("y_pred.csv", index=False)
print(y_pred_df.head())

Conclusion:

This project involved building a deep learning model to classify various types of crimes using the UCF Crime Dataset. 

I first imported necessary libraries and set up paths for training and test data, then loaded and visualized example images from the dataset to understand the data structure. I then used ImageDataGenerator for preprocessing the images, including rescaling, normalization, and augmentation techniques like flipping.

For the model architecture, I defined a Convolutional Neural Network (CNN) with multiple layers, including convolutional, batch normalization, ReLU activation, max-pooling, and dropout layers to prevent overfittingm then compiled the model using the Adam optimizer and categorical cross-entropy loss function.

For training, I made configured callbacks for early stopping and learning rate reduction based on validation accuracy, then trained the model on the training dataset and evaluated it using the validation dataset, which involved predicting the classes for the test dataset and calculating the accuracy, which was approximately 67.66%.

Next I lotted the ROC curve and calculated the Area Under the Curve (AUC) to measure the model's performance, resulting in an AUC of approximately 0.503, and then plotted the precision-recall curve and calculated the average precision score, which was approximately 0.677.

The model achieved a test accuracy of around 67.66%, indicating a reasonable performance given the complexity of the task and the diversity of the dataset. The ROC and precision-recall curves provided insights into the model's performance across different thresholds.

To improve the model's performance, I am considering experimenting with deeper architectures, more advanced data augmentation techniques, and fine-tuning pre-trained models.
Additionally, I will be exploring different hyperparameters and training for more epochs.
The predictions were saved to a CSV file, providing a record of the predicted labels for the test dataset.